In [1]:
import tweepy
from kafka import KafkaProducer
import json
import time
import random

<br>

In [2]:
class Main_Producer:
    
    def __init__(self, host="localhost", port="9092", topic_name="streamtest", interval=10, value_serializer=False):
        if not value_serializer:
            self.producer = KafkaProducer(bootstrap_servers=[host+":"+port])
        else:
            self.producer = KafkaProducer(bootstrap_servers=[host+":"+port],value_serializer=lambda x: json.dumps(x).encode('utf-8'))
        self.topic = topic_name       
        self.interval = interval
        self._connect_to_tweetAPI_v2()

    def _connect_to_tweetAPI_v2(self):
        # twitter setup 
        consumer_key = "2t7122v3yHERnEFHjA1XzWuQK"
        consumer_secret = "x6Q1cJISzURekVt2R8ra6NS6Teu6Ro7ZFvhDtuIxHmjZtUF9vV"
        access_token = "1445105220300705796-WXJTVdxvGLtX3TzjuxSAVMOMJJykzd"
        access_token_secret = "NZC2kf0QySMky4xKufwB4hS2cmr2c00CFWH3zjG13U3An"

        try:
            # Creating the authentication object
            auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
            # Setting your access token and secret
            auth.set_access_token(access_token, access_token_secret)
            # Creating the API object by passing in auth information
            self.api = tweepy.API(auth)
            print("connected to tweepy API...\n")
        except Exception as e:
            print(str(e))

    def _send(self, record):
        try:
            self.producer.send(self.topic,  record.encode('utf-8'))
        except Exception as e:
            print(str(e))

    def _get_twitter_data(self):
        query= [ 
                   [51.5522, 0.153621],[51.53744, 0.080059],[51.3475, -0.196499],[51.536207,-0.033004],
                   [51.487648, -0.142261],[51.48758, -0.139138],[51.488704, 0.046034],[51.5522,0.153621],
                   [51.54511,0.165546],[51.536326,0.076951],[51.535612,0.079154],[51.53744,0.080059],
                   [51.537106,0.077044],[51.535612,0.079154],[51.539801,0.132861],[51.53261,0.132683],
                   [51.529235,0.104454],[51.613432,-0.248606],[51.603441,-0.253957],[51.605284,-0.174721],
                   [51.592176,-0.212927],[51.556199,-0.213587],[51.556919,-0.2148],[51.478322,0.179518]
                ]
        res = self.api.search_tweets(q ="#", lang='en-us', result_type="recent")
        for i in res:
            if i.coordinates is  None:
                tweet = {
                      "id": str(i.user.id_str ) ,
                      "timestamp": str(i.created_at),
                      "coordinates": str(" { 'type': 'Point', 'coordinates': " )+str(random.choice(query) ) + str("}"),
                      "text": str(i.text),
                
                     }     
                record = json.dumps(tweet)
                print(record)
                print("-------------------------------")
                self._send(record)

    def periodic_work(self):
        self._get_twitter_data()
        time.sleep(self.interval)

<br>

In [3]:
producer_0 = Main_Producer()

while(True):
    producer_0.periodic_work()

connected to tweepy API...

{"id": "1307744233936293890", "timestamp": "2023-01-04 16:37:15+00:00", "coordinates": " { 'type': 'Point', 'coordinates': [51.488704, 0.046034]}", "text": "@kingkong45_eth @DeeboGoblin @icecreamsandwtf @Pak_Gregg i am in Lisbon which means i should be added \ud83d\ude01"}
-------------------------------
{"id": "1476496647677005826", "timestamp": "2023-01-04 16:37:15+00:00", "coordinates": " { 'type': 'Point', 'coordinates': [51.53744, 0.080059]}", "text": "RT @JoyisBackAgain: The Order         What I delivered \ud83d\ude0d\u2764\ufe0f https://t.co/4njVupJLr8"}
-------------------------------
{"id": "1218291605058154496", "timestamp": "2023-01-04 16:37:15+00:00", "coordinates": " { 'type': 'Point', 'coordinates': [51.537106, 0.077044]}", "text": "RT @riminatozaki: wts lfb ph\nbetween 1&amp;2 pcs\n\u2014 \u20b185 ea\n\u2014 payo\n\u2014 onhand\n\u2014 not for sensitive buyers\n\u2014 mop: gcash\n\u2014 mod: ggx direct \n\u2014 loc: cav\u2026"}
---------------

KeyboardInterrupt: 